In [1]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from IPython.display import Markdown as md
import os
import ipywidgets as wg
from IPython.display import display
from matplotlib import cm
import math
import json
from random import randint
import sys
sys.path.append('../src/')
import importlib
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import visualizations
importlib.reload(visualizations)
from visualizations import *
import detection
importlib.reload(detection)
from detection import *

## Parâmetros

In [2]:
proto_file = "../Models/Openpose/coco/pose_deploy_linevec.prototxt"
weights_file = "../Models/Openpose/coco/pose_iter_440000.caffemodel"
videos_dir = "../Videos/"
data_dir = "../Data/"

n_points = 18
# COCO Output Format
keypoints_mapping = np.array(['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                    'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 
                    'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear'])

pose_pairs = np.array([[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],
              [1,8], [8,9], [9,10], [1,11], [11,12], [12,13],
              [1,0], [0,14], [14,16], [0,15], [15,17],
              [2,17], [5,16]])

# index of pafs correspoding to the pose_pairs
# e.g for POSE_PAIR(1,2), the PAFs are located at indices (31,32) of output, Similarly, (1,5) -> (39,40) and so on.
map_idx = np.array([[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], 
          [19,20], [21,22], [23,24], [25,26], [27,28], [29,30], 
          [47,48], [49,50], [53,54], [51,52], [55,56], 
          [37,38], [45,46]])

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

## Functions

In [45]:
def VisualizeHeatmap(image, output, j, threshold=0.1, alpha=0.6, binary=False):
    frame_width = image.shape[1]
    frame_height = image.shape[0]
    prob_map = output[0, j, :, :]
    prob_map = cv2.resize(prob_map, (frame_width, frame_height))
    if(binary == False):
        prob_map = np.where(prob_map<threshold, 0.0, prob_map)
    else:
        prob_map = np.where(prob_map<threshold, 0.0, prob_map.max())
    plt.figure(figsize=[14,10])
    plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
    plt.imshow(prob_map, alpha=alpha, vmax=prob_map.max(), vmin=0.0)
    plt.colorbar()
    plt.axis("off")

In [35]:
def SingleFrameInference(frame, print_time=True):
    t = time.time()
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
    
    frame_width = frame.shape[1]
    frame_height = frame.shape[0]
    
    # Fix the input Height and get the width according to the Aspect Ratio
    in_height = 368
    in_width = 368

    #inp_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (in_width, in_height),
    #                          (0, 0, 0), swapRB=False, crop=False)

    inp_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (in_width, in_height),
                              (0, 0, 0), swapRB=False, crop=False)
    
    net.setInput(inp_blob)
    net_output = net.forward()
    time_taken = time.time() - t
    if print_time:
        print("Time Taken = {}".format(time_taken))
    return net_output, time_taken, inp_blob

In [14]:
def inferenceOP(video_name, n, threshold=0.1, n_interp_samples=10, paf_score_th=0.1, conf_th=0.7):
    
    image, frame_width, frame_height = getFrame(video_name, n)
    output, _ = singleFrameInference(image, print_time=False)

            # Gerar lista classificando os pontos para cada pessoa identificada
    personwise_keypoints, keypoints_list = keypointsFromHeatmap(output, frame_width, frame_height, threshold, n_interp_samples, paf_score_th, conf_th)
    return personwise_keypoints, keypoints_list

In [24]:
def visualizeResult(frame, personwise_keypoints, keypoints_list, persons, joint_pairs):
    
    frame_out = frame.copy()
    
    if (persons[0] == -1):
        persons = np.arange(len(personwise_keypoints))
        
    if (joint_pairs[0] == -1):
        joint_pairs = np.arange(len(pose_pairs)-2)
        
    for i in joint_pairs:
        for n in persons:
            index = personwise_keypoints[n][np.array(pose_pairs[i])]
            if -1 in index:
                continue
            B = np.int32(keypoints_list[index.astype(int), 0])
            A = np.int32(keypoints_list[index.astype(int), 1])
            cv2.line(frame_out, (B[0], A[0]), (B[1], A[1]), colors[i], 3, cv2.LINE_AA)

    plt.figure(figsize=[8,6])
    plt.imshow(frame_out[:,:,[2,1,0]])
    plt.axis("off")

## Teste

In [8]:
videos = os.listdir(videos_dir)
video_dropdown = wg.Dropdown( options=videos,
                        value='Victor.mp4',
                        description='Video:',
                        disabled=False)
frame_n= wg.IntText(value=0,description='Frame:')
y = wg.interactive_output(visualizeFrame, {"video_name":video_dropdown, "n":frame_n})
hbox = wg.HBox([video_dropdown, frame_n])
display(hbox, y)

Output()

In [27]:
personwise_keypoints, keypoints_list = inferenceOP(video_dropdown.value, frame_n.value)

In [28]:
def interactiveResult(persons, joint_pairs):
    visualizeResult(image1, personwise_keypoints, keypoints_list, persons, joint_pairs)

persons = wg.SelectMultiple(options=[i for i in range(len(personwise_keypoints))],
                            value=[i for i in range(len(personwise_keypoints))],
                            rows=len(personwise_keypoints),
                            description='Persons',
                            disabled=False)

joint_pairs = wg.SelectMultiple(options=[i for i in range(len(pose_pairs))],
                                value=[i for i in range(len(pose_pairs))],
                                rows=5,
                                description='Pose Pairs',
                                disabled=False)

hbox_res = wg.HBox([persons, joint_pairs])

i_res = wg.interactive_output(interactiveResult, {"persons": persons,
                                                  "joint_pairs": joint_pairs})

display(hbox_res, i_res)

Output()

In [48]:
video_name = video_dropdown.value
image1, frame_width, frame_height = getFrame(video_name, frame_n.value)
nn_output, _, _ = SingleFrameInference(image1)

Time Taken = 3.4008071422576904


In [46]:
def getHeatmap(joint_n, threshold, alpha, binary):
    VisualizeHeatmap(image=image1, output=output, j=joint_n, 
                      threshold=threshold, alpha=alpha, binary=binary)
    
joint_n = wg.IntText(value=0, description='Joint:',
                     layout=wg.Layout(display='flex',flex_flow='line',align_items='flex-start',
                                      justify_content='flex-start',width='18%'))
thres = wg.FloatText(value=0.1, step=0.1, description='Threshold:',
                     layout=wg.Layout(display='flex',flex_flow='line',align_items='flex-start',
                                      justify_content='flex-start',width='18%'))
alpha = wg.FloatText(value=0.6, step=0.1, description='Alpha:',
                     layout=wg.Layout(display='flex',flex_flow='line',align_items='flex-start',
                                      justify_content='flex-start',width='18%'))
show_points = wg.Checkbox(value=False, description='Points',disabled=False,
                     layout=wg.Layout(display='flex',flex_flow='line',align_items='flex-start',
                                      justify_content='flex-start',width='18%'))
show_heatmap = wg.Checkbox(value=False, description='Heatmap',disabled=False,
                     layout=wg.Layout(display='flex',flex_flow='line',align_items='flex-start',
                                      justify_content='flex-start',width='30%'))
binary = wg.Checkbox(value=False, description='Binary',disabled=False,
                     layout=wg.Layout(display='flex',flex_flow='line',align_items='flex-start',
                                      justify_content='flex-start',width='18%'))
i_heatmap = wg.interactive_output(getHeatmap,
            {"joint_n": joint_n,
            "threshold": thres,
            "alpha": alpha,
            "binary": binary})

ht_hbox_0 = wg.HBox([show_points, show_heatmap],                     
                    layout=wg.Layout(display='flex',flex_flow='line',
                                     align_items='flex-start',justify_content='flex-start'))
ht_hbox_1 = wg.HBox([joint_n, thres])
ht_hbox_2 = wg.HBox([alpha, binary])
ht_vbox_1 = wg.VBox([ht_hbox_0, ht_hbox_1, ht_hbox_2], 
                    layout=wg.Layout(border='solid 2px', display='flex'))
display(ht_vbox_1, i_heatmap)

Output()

In [49]:
def getKeypointsList(output, frame_width, frame_height, threshold=0.1, print_list=False):
    detected_keypoints = []
    keypoints_list = np.zeros((0,3))
    keypoint_id = 0
    for part in range(n_points):
        prob_map = output[0,part,:,:]
        prob_map = cv2.resize(prob_map, (frame_width, frame_height))
        keypoints = getKeypoints(prob_map, threshold)
        if print_list:
            print("Keypoints - {} : {}".format(keypoints_mapping[part], keypoints))
        keypoints_with_id = []
        for i in range(len(keypoints)):
            keypoints_with_id.append(keypoints[i] + (keypoint_id,))
            keypoints_list = np.vstack([keypoints_list, keypoints[i]])
            keypoint_id += 1

        detected_keypoints.append(keypoints_with_id)
    
    return keypoints_list, detected_keypoints

In [50]:
# Find the Keypoints using Non Maximum Suppression on the Confidence Map
def getKeypoints(prob_map, threshold=0.1):
    
    map_smooth = cv2.GaussianBlur(prob_map,(3,3),0,0)

    map_mask = np.uint8(map_smooth>threshold)
    keypoints = []
    
    #find the blobs
    _, contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #for each blob find the maxima
    for cnt in contours:
        blob_mask = np.zeros(map_mask.shape)
        blob_mask = cv2.fillConvexPoly(blob_mask, cnt, 1)
        masked_prob_map = map_smooth * blob_mask
        _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)
        keypoints.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))

    return keypoints

In [51]:
keypoints_list, detected_keypoints = getKeypointsList(nn_output, frame_width, frame_height, threshold=0.1, print_list=False)

In [54]:
valid_pairs, invalid_pairs = getValidPairs(output, detected_keypoints)
personwise_keypoints = getPersonwiseKeypoints(valid_pairs, invalid_pairs)

IndexError: list index out of range

## Caffe

In [ ]:
proto_file = "/home/victormacedo10/TCC_Biomec/Models/Openpose/coco/pose_deploy_linevec_5_stages.prototxt"
weights_file = "/home/victormacedo10/TCC_Biomec/Models/Openpose/coco/pose_iter_440000.caffemodel"

In [34]:
proto_file = "../Models/Openpose/coco/pose_deploy_linevec_5_stages.prototxt"
weights_file = "../Models/Openpose/coco/pose_iter_440000.caffemodel"

In [35]:
net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
image, _, _ = getFrame('Victor.mp4', 0)
in_height = 368
in_width = 368
inp_blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (in_width, in_height),
                          (0, 0, 0), swapRB=False, crop=False)
net.setInput(inp_blob)
#net_output = net.forward()

error: OpenCV(3.4.3) /home/victormacedo10/opencv-3.4.3/modules/dnn/src/caffe/caffe_io.cpp:1151: error: (-2:Unspecified error) FAILED: ReadProtoFromTextFile(param_file, param). Failed to parse NetParameter file: ../Models/Openpose/coco/pose_deploy_linevec_5_stages.prototxt in function 'ReadNetParamsFromTextFileOrDie'


In [30]:
personwise_keypoints, keypoints_list = inferenceOP('Victor.mp4', 0)

In [25]:
def interactiveResult(persons, joint_pairs):
    visualizeResult(image, personwise_keypoints, keypoints_list, persons, joint_pairs)

persons = wg.SelectMultiple(options=[i for i in range(len(personwise_keypoints))],
                            value=[i for i in range(len(personwise_keypoints))],
                            rows=len(personwise_keypoints),
                            description='Persons',
                            disabled=False)

joint_pairs = wg.SelectMultiple(options=[i for i in range(len(pose_pairs))],
                                value=[i for i in range(len(pose_pairs)-1)],
                                rows=5,
                                description='Pose Pairs',
                                disabled=False)

hbox_res = wg.HBox([persons, joint_pairs])

i_res = wg.interactive_output(interactiveResult, {"persons": persons,
                                                  "joint_pairs": joint_pairs})

display(hbox_res, i_res)

Output()

In [31]:
def interactiveResult(persons, joint_pairs):
    visualizeResult(image, personwise_keypoints, keypoints_list, persons, joint_pairs)

persons = wg.SelectMultiple(options=[i for i in range(len(personwise_keypoints))],
                            value=[i for i in range(len(personwise_keypoints))],
                            rows=len(personwise_keypoints),
                            description='Persons',
                            disabled=False)

joint_pairs = wg.SelectMultiple(options=[i for i in range(len(pose_pairs))],
                                value=[i for i in range(len(pose_pairs)-1)],
                                rows=5,
                                description='Pose Pairs',
                                disabled=False)

hbox_res = wg.HBox([persons, joint_pairs])

i_res = wg.interactive_output(interactiveResult, {"persons": persons,
                                                  "joint_pairs": joint_pairs})

display(hbox_res, i_res)

Output()